# Merging EBMs

In this notebook, we will fit several Explainable Boosting Machines (EBMs) and then merge them into a single EBM model. The resulting EBM model retains the glassbox interpretability of the component EBM models, allowing us to view both global and local explanations of the merged model.

This notebook can be found in our [**_examples folder_**](https://github.com/interpretml/interpret/tree/develop/docs/interpret/python/examples) on GitHub.

In [ ]:
# install interpret if not already installed
try:
    import interpret
except ModuleNotFoundError:
    !pip install --quiet interpret pandas scikit-learn

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from interpret import show

from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

seed = 42
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

<h2>Explore the dataset</h2>

In [ ]:
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')
show(hist)

<h2>Training multiple EBM models</h2>

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier

# Fitting multiple EBM models with different training datasets and random seeds
seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
ebm1 = ExplainableBoostingClassifier(random_state=seed)
ebm1.fit(X_train, y_train)  

seed +=10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
ebm2 = ExplainableBoostingClassifier(random_state=seed)
ebm2.fit(X_train, y_train)  

seed +=10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
ebm3 = ExplainableBoostingClassifier(random_state=seed)
ebm3.fit(X_train, y_train)

In [ ]:
show(ebm1.explain_global(name='EBM 1'))

<h2>Merging multiple trained EBM models</h2>

In [ ]:
from interpret.glassbox import merge_ebms

merged_ebm = merge_ebms([ebm1, ebm2 , ebm3])
show(merged_ebm.explain_global(name='Merged EBM'))